<h1> Define datasets to incorporate </h1>

In [ ]:
pen_id_time_ranges = [
    {
        'pen_id': 4,
        'start_date': '2020-07-12',
        'end_date': '2020-07-19',
        'tags': ['low_turbidity']
    },
    {
        'pen_id': 173,
        'start_date': '2021-01-01',
        'end_date': '2021-01-15',
        'tags': ['low_turbidity', 'small_fish']
    },
    {
        'pen_id': 194,
        'start_date': '2021-01-01',
        'end_date': '2021-01-20',
        'tags': ['low_turbidity']
    },
    {
        'pen_id': 56,
        'start_date': '2020-07-06',
        'end_date': '2020-07-14',
        'tags': ['medium_turbidity']
    },
    {
        'pen_id': 100,
        'start_date': '2020-07-13',
        'end_date': '2020-07-21',
        'tags': ['medium_turbidity']
    },
    {
        'pen_id': 86,
        'start_date': '2020-06-10',
        'end_date': '2020-06-12',
        'tags': ['medium_turbidity', 'trout']
    },
    {
        'pen_id': 95,
        'start_date': '2020-06-20',
        'end_date': '2020-06-26',
        'tags': ['high_turbidity']
    },
    {
        'pen_id': 4,
        'start_date': '2020-06-13',
        'end_date': '2020-06-18',
        'tags': ['high_turbidity']
    },
    {
        'pen_id': 114,
        'start_date': '2020-07-12',
        'end_date': '2020-07-22'
    } 
]



<h1> For each dataset, construct stitched paired images and upload to s3. Prep for PLALI insertion </h1>

In [ ]:
def get_dw_biomass_data(pen_id, start_date, end_date):
    query = """
        SELECT * 
        FROM prod.biomass_computations
        WHERE pen_id={0} 
        AND captured_at BETWEEN {1} and {2}
        AND date_part('hour', captured_at) >= {3} 
        AND date_part('hour', captured_at) <= {4}
    """.format(pen_id, start_date, end_date, 7, 15)
    
    df = rds.extract_from_database(query)
    return df
    

    

In [ ]:
def get_512_512_dataset(df):
    
    full_frame_to_crop_urls = defaultdict(list)
    for idx, row in df.iterrows():
        left_crop_url = row.left_crop_url
        left_full_frame_512_url = get_full_frame_512_url(left_crop_url)
        full_frame_to_crop_urls[left_full_frame_512_url].append(row.left_crop_url)
        full_frame_to_crop_urls[left_full_frame_512_url].append(row.right_crop_url)
        
    left_image_urls, right_image_urls, counts = []
    for k, v in full_frame_to_crop_urls.items():
        left_image_urls.append(k)
        right_image_urls.append(k.replace('left', 'right'))
        counts.append(len(v))
    
    df_512 = pd.DataFrame({
        'left_image_url': left_image_urls,
        'right_image_url': right_imgae_urls,
        'count': counts
    })
    
    return df_512
    
        


In [ ]:
def get_paired_urls(dataset, count):
    
    # get paired images that are guaranteed to have crops
    pen_id, start_date, end_date = dataset['pen_id'], dataset['start_date'], dataset['end_date']
    df_dw = get_dw_biomass_data(pen_id, start_date, end_date)
    df_512 = get_512_512_dataset(df_dw)
    df_512 = df_512.sort_values('count', ascending=False)
    high_density_left_urls = df_512.left_image_url.head(500)
    right_density_left_urls = df_512.right_image_url.head(500)
    high_density_paired_urls = list(zip(high_density_left_urls, high_density_right_urls))
    
    # get paired images from different density classes
    get_all_s3_keys(pen_id, start_date, end_date)
    
    
    
    
    
    


<h1> Construct dataset for Hungarian Matching evaluation </h1>

In [ ]:
for dataset in pen_id_time_ranges:
    
    df = pd.DataFrame()
    
    paired_urls = get_paired_urls(dataset, 1000)
    left_image_urls, right_image_urls, stiched_image_urls = [], [], []
    metadatas = []
    for paired_url in paired_urls:
        left_image_url, right_image_url = paired_url
        stitched_image_f = generate_stitched_image(left_image_url, right_image_url )
        bucket, key = 'aquabyte-images-adhoc', 'alok/O2Kr2_dataset/{}'.format(os.path.basename(stitched_image_f))
        s3.upload_file(stitched_image_f, bucket, key)
        stitched_image_url = os.path.join(prefix, bucket, key)
        
        metadata = dict(dataset)
        metadata['left_image_url'] = left_image_url
        metadata['right_image_url'] = right_image_url
        metadatas.append(metadata)
        
    
    df['metadata'] = metadatas
    df['url'] = stitched_image_urls
    
        
    
    